Project begins here. Import required libraries/functions

In [72]:
import pandas as pd
import numpy as np
import re, math, hashlib, ast, sys, io, random
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Import raw data in csv file.

In [73]:
posts_df = pd.read_csv("Twitter_Analysis.csv")
print(raw_df.head())


   id                                               post  true  \
0   0  @POTUS Biden Blunders - 6 Month Update\n\nInfl...     1   
1   1  @S0SickRick @Stairmaster_ @6d6f636869 Not as m...     1   
2   2  THE SUPREME COURT is siding with super rich pr...     1   
3   3  @POTUS Biden Blunders\n\nBroken campaign promi...     1   
4   4  @OhComfy I agree. The confluence of events rig...     1   

                                        cleaned_post  
0  biden blunders 6 month update inflation delta ...  
1  not as many people are literally starving and ...  
2  the supreme court is siding with super rich pr...  
3  biden blunders broken campaign promises inflat...  
4  i agree the confluence of events right now is ...  


Isolate selected columns in the order of id, post, true/false. Remove any rows that are missing either the post or true/false data.


In [74]:
id_column = 'id'
post_column = 'tweet'
truth_column = 'majority_target'

def convert_true_false_to_bool(true_false):
        if true_false:
            return 1
        else:
            return 0
        
posts_df[truth_column] = posts_df[truth_column].apply(convert_true_false_to_bool)

posts_df = posts_df[[id_column, post_column, truth_column]]
posts_df.columns = ['id', 'post', 'true']
posts_df = posts_df.dropna(subset=['post', 'true'])
print(posts_df.head())

   id                                               post  true
0   0  @POTUS Biden Blunders - 6 Month Update\n\nInfl...     1
1   1  @S0SickRick @Stairmaster_ @6d6f636869 Not as m...     1
2   2  THE SUPREME COURT is siding with super rich pr...     1
3   3  @POTUS Biden Blunders\n\nBroken campaign promi...     1
4   4  @OhComfy I agree. The confluence of events rig...     1


Normalize text to remove urls, casing, and other symbols.

In [75]:
def clean_text_basic(text: str) -> str:
    """Basic, OPTIONAL cleaner for social/email text. Keeps semantics coarse."""
    if not isinstance(text, str):
        return ""
    t = re.sub(r"http\S+","", text)        # remove URLs
    t = re.sub(r"@\w+","", t)              # remove mentions
    t = t.replace("#","")                  # remove hashtag symbol (keep words)
    t = re.sub(r"[^\w\s]"," ", t)          # strip heavy punctuation
    t = re.sub(r"\s+"," ", t).strip().lower()
    return t

posts_df['cleaned_post'] = posts_df['post'].apply(clean_text_basic)
posts_df[['post','cleaned_post','true']].head(5)

,post,cleaned_post,true
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,biden blunders 6 month update inflation delta ...,1
1,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,not as many people are literally starving and ...,1
2,THE SUPREME COURT is siding with super rich pr...,the supreme court is siding with super rich pr...,1
3,@POTUS Biden Blunders\n\nBroken campaign promi...,biden blunders broken campaign promises inflat...,1
4,@OhComfy I agree. The confluence of events rig...,i agree the confluence of events right now is ...,1
